In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
d = [[1,2],[1,4],[5,1]]
ff = np.array(d)
indexes = np.where(ff == 1)
print(indexes)
tup = np.array(list(zip(indexes[0], indexes[1])))
print(tup[indexes])


(array([0, 1, 2]), array([0, 0, 1]))
[0 1 1]


In [ ]:
class Grid:
  def __init__(self, N, input_dim, output_dim, time_initial = 500, time_interval = 100,time_steady = 40, flow_blocked_percentage_initial = 0.1):
    """Arguments:
          self: instance of class
          N: number of cells in a row, N x N
          input_dim: number of entreis in input vector
          output_dim: number of entries in output vector
          time_initial: number of steps of the system before beginning to determine probability distribution
          time_interval: number of steps over which you check to see if a cell was occupied at some time
          time_steady: number of time_interval units
          flow_blocked_percentage_initial: percentage of cells for which flow is blocked
        Returns:
          Nothing"""
    self.N = N # create instance variable
    #self.grid = np.random.choice([0,1], (self.N, self.N)) # 0 means flow permitted, 1 means flow blocked
    flow_blocked_initial_number = round((N**2) * flow_blocked_percentage_initial)
    self.grid = np.array([1] * flow_blocked_initial_number + [0] * ((N**2) - flow_blocked_initial_number))
    np.random.shuffle(self.grid) # mix the order of walls up
    self.grid = np.reshape(self.grid, (self.N, self.N))
    # select input_dim values from the zero (open) posiitons
    zero_indexes = np.where(self.grid == 0) # find location of open cells
    zero_indexes = np.array(list(zip(zero_indexes[0], zero_indexes[1])))
    IO_indexes = np.random.choice(range(len(zero_indexes)), input_dim + output_dim, replace=False)
    self.input_indexes = zero_indexes[IO_indexes[0:input_dim]]
    self.output_indexes = zero_indexes[IO_indexes[input_dim: input_dim + output_dim]]
    self.base_particle_array = [] # array of Base_Particle objects
    self.state = np.zeros((N,N)) # number of particles at each grid position
    self.input_dim = input_dim # input dimension
    self.output_dim = output_dim # output dimension
    self.time_initial = time_initial
    self.time_interval = time_interval
    self.time_steady = time_steady
  def Grid_set_inputs(self, input_vector):
    """Arguments:
          self: instance of class
          input_vector: binary vector of size input_dim x 1 representing input to system
      Returns:
          Nothing"""
    self.base_particle_array = [] # empty state
    for input_element, input_index in zip(input_vector, self.input_indexes): #set each input state to corresponding input vector element
      #self.state[input_index[0]][input_index[1]] = input_element
      if input_element == 1: # only create particle if there is a 1
        self.base_particle_array.append(Base_Particle(input_index[0], input_index[1], self.N))
    self.Grid_update_state() # update internal state
      
  def Grid_update_state(self):
    """Arguments:
          self: instance of class
       Returns:
          Nothing (updates number of particles in each cell)"""
    self.state = np.zeros((self.N, self.N)) # initially empty
    for base_particle in self.base_particle_array: # iterate through all particles
      x, y = base_particle.Base_Particle_get_position()
      #self.state[x][y] = self.state[x][y] + 1
      self.state[x][y] = 1
  
  def Grid_run_simulation(self):
    """Arguments:
          self: instance of class
       Returns:
          Nothing"""
    # 3d matrix for state of system at different points in time
    grid_interval_unit = np.zeros((N,N,self.time_interval))
    grid_steady = np.zeros((N,N,self.time_steady))
    for step in range(self.time_initial + self.time_steady * self.time_interval):
      for base_particle in self.base_particle_array: # iterate over all Base_Particle objects
        x,y = base_particle.Base_Particle_get_position()
        # initialize all directions of motion to be possible
        dx_pos = True
        dx_neg = True
        dy_pos = True
        dy_neg = True
        # corner cases (actually)
        # check if dx_pos should be False
        # if x == self.N - 1:
        #   dx_pos = False
        if self.grid[(x + 1) % self.N][y] == 1:
          dx_pos = False
        # check if dx_neg should be False
        # if x == 0:
        #   dx_neg = False
        if self.grid[(x - 1 + self.N) % self.N][y] == 1:
          dx_neg = False
        # check if dy_pos should be False
        # if y == self.N - 1:
        #   dy_pos = False
        if self.grid[x][(y + 1) % self.N] == 1:
          dy_pos = False
        # check if dy_neg should be false
        # if y == 0:
        #   dy_neg = False
        if self.grid[x][(y - 1 + self.N) % self.N] == 0:
          dy_neg = False
        
        # set boundary of particle
        base_particle.Base_Particle_set_boundary(dx_pos, dx_neg, dy_pos, dy_neg)
        base_particle.Base_Particle_update_position() # move the particle
      if step >= self.time_initial: # in the steady state
        grid_interval_unit[:,:,(step - self.time_initial) % self.time_interval] = self.state.copy()
        if (step - self.time_initial) % self.time_interval == 0: # end of interval reached
          interval_result = np.sum(grid_interval_unit, axis = 2)
          interval_result[np.nonzero(interval_result)] = 1
          grid_steady[:,:, int((step - self.time_initial) / self.time_interval)] = interval_result
      self.Grid_update_state()
    # update the state
    #self.Grid_update_state()
    # compute the steady state probabilities/frequencies
    #grid_across_time[np.nonzero(grid_across_time)] = 1
    self.steady_state_distribution = np.sum(grid_steady, axis = 2) / self.time_steady

  def Grid_train(self, X_train, Y_train, eroding_factor = 0.3):
    self.Grid_set_inputs(X_train) # set inputs
    self.Grid_run_simulation() # run a pass through

    # compute error at each output entry
    error_vector = np.zeros(self.output_dim)
    for output_index, i in zip(self.output_indexes, range(self.output_dim)): # iterate throuhg the output indexes
      error_vector[i] = Y_train[i] - self.steady_state_distribution[output_index[0]][output_index[1]] 
    print("error vector:", error_vector)
    # create eroding particles
    eroding_particle_array = []
    # locate places to insert eroding particles, all places with a positive error for now
    eroding_particle_indexes = self.output_indexes[np.argwhere(error_vector > 0)]
    print("eroding indexes", eroding_particle_indexes)
    # only insert a select number of eroding particles, with the probability of an eroding particle being inserted proportional to the error
    error_vector_positive = error_vector[np.argwhere(error_vector > 0)]
    num_positive_errors = len(error_vector_positive) 
    eroding_particle_indexes = eroding_particle_indexes[np.random.choice(range(num_positive_errors), 1 + int(eroding_factor * num_positive_errors), error_vector_positive)]
    print("eroding indexes", eroding_particle_indexes)
    for eroding_particle_index in eroding_particle_indexes[0]: # loop through all eroding particle indexes that have been selected
      eroding_particle_array.append(Base_Particle(eroding_particle_index[0], eroding_particle_index[1], self.N))
    

    # create depositing particles
    # locate places to insert depositing particles, all places with a negative error for now
  def Grid_get_steady_state_distribution(self):
    """Arguments:
          self: instance of class
       Returns:
          self.steady_state_distribution: average number of base particles per cell during time_steady"""
    return self.steady_state_distribution     
  def Grid_get_output_vector(self):
    """Arguments:
          self: instance of class
       Returns:
          output_vector: state of system at output indexes"""
    output_vector = np.zeros(self.output_dim)
    for output_index, i in zip(self.output_indexes, range(self.output_dim)): # iterate throuhg the output indexes
      output_vector[i] = self.state[output_index[0]][output_index[1]]
    return output_vector

  
  def Grid_generate_heat_map(self):
    """Arguments:
          self: instance of class
       Returns:
          Nothing (displays a heat map)"""
    labels = np.zeros((8,12))
    fig, ax = plt.subplots(figsize = (12,7))
    title = "Base Particle Positions"
    plt.title(title, fontsize = 20)
    ttl = ax.title
    sns.heatmap(self.state, fmt = "", cmap = "hot_r", linewidths = 0.30, ax = ax)
  

  def Grid_get_grid(self):
    """Arguments:
          self: instance of class
        Returns:
          self.grid: array of which cells are open"""
    return self.grid

  def Grid_get_input_indexes(self):
    """Arguments:
          self: instance of class
      Returns:
          self.input_indexes: indexes that are inputs to system"""
    return self.input_indexes
  
  def Grid_get_output_indexes(self):
    """Arguments:
          self: instance of class
        Returns:
          self.output_indexes: indexes that are outputs to system"""
    return self.output_indexes


In [ ]:
STAY = 0
DX_POS = 1
DX_NEG = 2
DY_POS = 3
DY_NEG = 4
class Base_Particle:
  def __init__(self, x, y, N):
    """Arguments:
          self: instance of class
          x: horizontal position in grid
          y: vertical position in grid
          N = grid side length
       Returns:
          Nothing"""
    self.x = x
    self.y = y
    self.N = N
    # initially can move in all directions
    self.dx_pos = True
    self.dx_neg = True
    self.dy_pos = True
    self.dy_neg = True

  def Base_Particle_set_boundary(self, dx_pos, dx_neg, dy_pos, dy_neg):
    """Arguments:
          self: instance of class
          dx_pos: position motion in x
          dx_neg: negative motion in x
          dy_pos: positive motion in y
          dy_neg: negative motion in y
      Returns:
          Nothing"""
    self.dx_pos = dx_pos
    self.dx_neg = dx_neg
    self.dy_pos = dy_pos
    self.dy_neg = dy_neg
  
  def Base_Particle_update_position(self):
    """Arguments:
          self: instance of class
       Returns:
          Nothing"""
    # compute probability of moving in each direction
    transition_probabilities = np.array([1, self.dx_pos, self.dx_neg, self.dy_pos, self.dy_neg])
    transition_probabilities = transition_probabilities / np.sum(transition_probabilities) 
    movement = np.random.choice(range(5), 1, p = transition_probabilities)
    # do nothing if movement == STAY
    if movement == DX_POS:
      self.x = self.x + 1
    if movement == DX_NEG:
      self.x = self.x - 1
    if movement == DY_POS:
      self.y = self.y + 1
    if movement == DY_NEG:
      self.y = self.y - 1
    
    # account for wrap around
    self.x = (self.x + self.N) % self.N
    self.y = (self.y + self.N) %self.N
  
  def Base_Particle_get_position(self):
    """Arguments:
          self: instance of class
       Returns:
          self.x: x-position
          self.y: y-position"""
    return (self.x, self.y)


    


In [ ]:
class Data:
  def __init__(self, input_vector_dim, input_vector_num_ones, output_vector_dim, num_training_examples, num_testing_examples):
    """Arguments:
          self: instance of class
          input_vector_dim: number of elements in input vector
          input_vector_num_ones: number of ones in the input vector
          output_vector_dim: number of elements in the output vector
          num_training_examples: number of examples used for training system
          num_testing_examples: number of examples for testing system
       Returns:
          Nothing"""
    self.input_vector_dim = input_vector_dim
    self.input_vector_num_ones = input_vector_num_ones
    self.output_vector_dim = output_vector_dim
    self.num_training_examples = num_training_examples
    self.num_testing_examples = num_testing_examples
    
    # generate the data
    X_template = np.array([1] * self.input_vector_num_ones + [0] * (self.input_vector_dim - self.input_vector_num_ones))
    # training data
    self.X_train = np.zeros((self.num_training_examples, self.input_vector_dim))
    self.Y_train = np.zeros((self.num_training_examples, self.output_vector_dim))
    for i in range(self.num_training_examples):
      self.X_train[i,:] = np.random.permutation(X_template)
      if self.X_train[i][0] == 0: # even
        self.Y_train[i,:] = np.array([1,0])
      if self.X_train[i][0] == 1: # odd
        self.Y_train[i,:] = np.array([0,1])
    # testing data
    self.X_test = np.zeros((self.num_testing_examples, self.input_vector_dim))
    self.Y_test = np.zeros((self.num_testing_examples, self.output_vector_dim))
    for i in range(self.num_testing_examples):
      self.X_test[i,:] = np.random.permutation(X_template)
      if self.X_test[i][0] == 0: # even
        self.Y_test[i,:] = np.array([1,0])
      if self.X_test[i][0] == 1: # odd
        self.Y_test[i,:] = np.array([0,1])

  def Data_get_data(self):
    """Arguments:
          self: instance of class
       Returns:
          self.X_train: X training vectors
          self.Y_train: Y training vectors
          self.X_test: X testing examples
          self.Y_test: Y testing examples"""
    return (self.X_train, self.Y_train, self.X_test, self.Y_test)

    


In [ ]:
N = 10
input_dim = 5
output_dim = 2
num_training_examples = 4
num_testing_examples = 6
g = Grid(N, input_dim, output_dim)
grid = g.Grid_get_grid()
print(grid)
print(g.Grid_get_input_indexes())
print(g.Grid_get_output_indexes())
# generate data
data = Data(input_dim, int(input_dim / 2), output_dim, num_training_examples, num_testing_examples)
X_train, Y_train, X_test, Y_test = data.Data_get_data()

# train model

g.Grid_train(X_train[0], Y_train[0])

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 1 0 0]
 [1 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]]
[[7 9]
 [9 9]
 [4 0]
 [0 3]
 [2 9]]
[[9 7]
 [1 5]]
error vector: [ 0.325 -0.775]
eroding indexes [[[9 7]]]
eroding indexes [[[9 7]]]


In [ ]:
N = 10
input_dim = 5
output_dim = 5
g = Grid(N, input_dim, output_dim)
grid = g.Grid_get_grid()

print(grid)
print(g.Grid_get_input_indexes())
print(g.Grid_get_output_indexes())
# input_vector = np.random.choice([0,1], input_dim) 
g.Grid_set_inputs(input_vector)
#g.Grid_generate_heat_map()
g.Grid_run_simulation()

#g.Grid_generate_heat_map()
#print(g.Grid_get_output_vector())
final = g.Grid_get_steady_state_distribution()
print(final)
print(np.max(final))


[[0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]]
[[4 9]
 [0 8]
 [2 2]
 [1 4]
 [2 9]]
[[9 2]
 [6 6]
 [9 3]
 [9 8]
 [4 8]]
[[0.0975 0.125  0.1975 0.2075 0.2425 0.2375 0.21   0.23   0.26   0.0325]
 [0.22   0.19   0.2075 0.06   0.2    0.2425 0.2575 0.2525 0.3075 0.2225]
 [0.235  0.235  0.2575 0.2    0.265  0.195  0.2275 0.215  0.2825 0.2   ]
 [0.1375 0.1675 0.1925 0.2375 0.2825 0.045  0.1575 0.1025 0.225  0.0425]
 [0.23   0.1775 0.1975 0.225  0.4225 0.2325 0.29   0.2325 0.31   0.1825]
 [0.175  0.175  0.19   0.225  0.4025 0.0275 0.1475 0.205  0.2375 0.055 ]
 [0.2475 0.2375 0.2025 0.225  0.3725 0.06   0.13   0.1525 0.25   0.2425]
 [0.31   0.2925 0.2675 0.2975 0.345  0.29   0.2625 0.175  0.2425 0.3175]
 [0.2575 0.2475 0.275  0.235  0.0725 0.2225 0.2225 0.22   0.245  0.345 ]
 [0.0425 0.135  0.2175 0.24   0.195  0.2

array([[1, 2],
       [3, 4],
       [5, 6]])

In [ ]:
r = np.array([[0,0,3],[4,5,6],[7,0,9],[0,0,12]])
print(r)
r[np.nonzero(r)] = 1
print(r)

[[ 0  0  3]
 [ 4  5  6]
 [ 7  0  9]
 [ 0  0 12]]
[[0 0 1]
 [1 1 1]
 [1 0 1]
 [0 0 1]]


In [ ]:
a = np.arr